In [1]:
import os, sys, shutil, glob, subprocess
import numpy as np
from string import template
from molSimplify.Classes.mol3D import *
from molSimplify.Classes.ligand import *
from molSimplify.Scripts.geometry import *
from molSimplify.Informatics.geo_analyze import * 
from molSimplify.Informatics.misc_descriptors import * 
from pymol.cgo import *
from pymol import cmd
from pymol import preset
from pymol import util

In [2]:
# paths
initialPath = "initial/initial2.xyz"
finalPath = "final/final2.xyz"
texSource = "../autotikz/"
reportPath = "reportName/"

# load in geos
finalMol = mol3D()
initialMol = mol3D()
initialMol.readfromxyz(initialPath)
finalMol.readfromxyz(finalPath)


if not os.path.isdir(reportPath):
    os.makedirs(reportPath)
filesToCopy = ['main','tikz_styles','header']
for files in filesToCopy:
    shutil.copy(texSource + files+'.tex',reportPath + files+'.tex')


In [ ]:
# def prepareAtomLine(symbol,coords):
#     # target = \drawFeAtom{5}{5}{-12}
#     drawType = str(symbol)+'Atom'
#     return "    \draw"+drawType +"{" + "}{".join([str(i) if abs(i) > 1E-3 else str(0) for i in coords]) + "}\n"

# def makePicture(linesToAdd,name,texSource,reportPath):
#     with open(reportPath + name +'.tex', 'w') as newf:
#         with open(texSource + "pictureTemplate.tex",'r') as ref:
#             reflines = ref.readlines()
#         for lines in reflines:
#             if lines.find("%% new stuff goes here!") == -1:
#                 newf.write(lines)
#             else:
#                 for newlines in linesToAdd:
#                     newf.write(newlines)
                    

In [ ]:
initialLiglist, initialLigdents, initialLigcons = ligand_breakdown(initialMol)
print(initialLigdents)
initailLigatoms = [len(i) for i in initialLiglist]  # number of atoms 
print(initialLiglist[0])
print(initialLigcons)

In [ ]:
def rot(initialMol, component):
    # component is 0,1,2 for x,y,z
    print('------Component: ' + str(component) + '------')
    maxComponent = 0
    Vect = []
    cInd = 0

    for conAtoms in initialLigcons:
        thisComponent = abs(initialMol.getAtom(conAtoms[0]).coords()[component])
        if maxComponent < thisComponent:
            Vect = initialMol.getAtom(conAtoms[0]).coords()
            cInd = conAtoms[0]
    
    vectSize = norm(Vect)
    if component == 0:
        target = [vectSize, 0, 0]
    elif component == 1:
        target = [0, vectSize, 0]
    elif component == 2:
        target = [0, 0, vectSize]
    
    thetaold = vecangle(Vect, target)
    
    theta, u = rotation_params(Vect,[0,0,0],target)
    new_mol = rotate_around_axis(initialMol,[0,0,0],u,-1*theta)
    
    VectNew = initialMol.getAtom(cInd).coords()
    print('\nafter rot the conatom is located at ' + str(VectNew))       
    print(VectNew) 
    print('\n')

    return new_mol, theta, u
    

In [ ]:
def orient(mol):
    new_mol, theta1, u1 = rot(mol, 2)
    new_mol, theta2, u2 = rot(new_mol, 1)
    new_mol, theta3, u3 = rot(new_mol, 0)
    return new_mol, [theta1, theta2, theta3], [u1, u2, u3]

In [ ]:
## initial geo parse
# spatialMax = [0,0,0]

# center on metal
initialMol.translate([-i for i in initialMol.getAtom(initialMol.findMetal()[0]).coords()])
finalMol.translate([-i for i in finalMol.getAtom(finalMol.findMetal()[0]).coords()])

print('######## INITIAL ########')
initialMolOrient, theta, u = orient(initialMol)
print('######## FINAL ########')
finalMolOrient = rotate_around_axis(finalMol,[0,0,0],u[0],-1*theta[0])
finalMolOrient = rotate_around_axis(finalMol,[0,0,0],u[1],-1*theta[1])
finalMolOrient = rotate_around_axis(finalMol,[0,0,0],u[2],-1*theta[2])
print('--reorientation done--')

initialMolOrient.writexyz('initial/initialOrient.xyz')
finalMolOrient.writexyz('final/finalOrient.xyz')

In [ ]:
# intial loop to get scaling
# for atom in new_mol.atoms:
#     x, y, z = atom.coords()
#     if abs(x) >= spatialMax[0]:
#         spatialMax[0] = abs(x)
#     if abs(y) >= spatialMax[1]:
#         spatialMax[1] = abs(y)
#     if abs(z) >= spatialMax[2]:
#         spatialMax[2] = abs(z)

# print(new_mol.coords())
# spatialScaling = max([8 / i for i in spatialMax])

# initialLines = []

# intial loop to get scaling
# for atom in new_mol.atoms:
#     initialLines.append(prepareAtomLine(atom.symbol(),[float(c)*spatialScaling for i,c in enumerate(atom.coords())]))
#     #initialLines.append(prepareAtomLine(atom.symbol(),atom.coords()))
    
# makePicture(linesToAdd = initialLines, name = "initialProc", texSource = texSource, reportPath = reportPath)
# makePicture(linesToAdd = initialLines, name = "finalProc",   texSource = texSource, reportPath = reportPath)

In [ ]:
curdir = os.getcurrentdir()
os.chdir(reportName)
cmd  = 'pdflatex main.tex'
p_sub = subprocess.Popen(cmd_str,shell=True,stdout=subprocess.PIPE)
os.chdir(curdir)

In [ ]:
def draw3d(load_path, save_path):
    cmd.delete('all')

    ## generate axes
    w = 0.06 # cylinder width 
    l = 3 # cylinder length
    h = 0.25 # cone hight
    d = w * 1.618 # cone base diameter

    # start coords, end coords, w is radius, color1 with grad to color2 in rgb
    obj = [CYLINDER, 0.0, 0.0, 0.0,   l, 0.0, 0.0, w, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0,
           CYLINDER, 0.0, 0.0, 0.0, 0.0,   l, 0.0, w, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0,
           CYLINDER, 0.0, 0.0, 0.0, 0.0, 0.0,   l, w, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0,
           CYLINDER, 0.0, 0.0, 0.0,   -l, 0.0, 0.0, w, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0,
           CYLINDER, 0.0, 0.0, 0.0, 0.0,   -l, 0.0, w, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0,
           CYLINDER, 0.0, 0.0, 0.0, 0.0, 0.0,   -l, w, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0,
          ]
    # load axes into picture
    cmd.load_cgo(obj, 'axes')

    # load structure and define properties 
    cmd.load(load_path)
    cmd.bg_color(color="white")
    util.cba
    preset.ball_and_stick(selection='all', mode=1)
    cmd.zoom('all', 1000, 0, 0)
    cmd.set('depth_cue', 0)

    # https://www.andre-gaschler.com/rotationconverter/
    view = [0.8535534, -0.3535534,  0.3826834,\
       0.5205981,  0.5497281, -0.6532815,\
       0.0205981,  0.7568349,  0.6532815,\
         0.000000000,    0.000000000,  -22.395387650,\
         0.000147820,    0.000066757,    0.000249624,\
        18.378971100,   26.411804199,  -25.000000000 ]

    cmd.set_view(view)
    cmd.png(save_path + '1.png', 500, 500, 300, 0, 0)
    
    #view2 from the other side
    for i in [2,3,4,5,6,7,8]:
        view[i] = view[i]*(-1)
    
    cmd.set_view(view)
    cmd.png(save_path + '2.png', 500, 500, 300, 0, 0)



In [ ]:
draw3d('initial/initialOrient.xyz', 'initial/initial_render')
draw3d('final/initialOrient.xyz', 'initial/initial_render')

In [46]:
-

getting lig assignment
[[2.45], [2.45]]
[[2.45], [2.45], [2.45], [2.45]]
getting lig assignment
[[2.377422038884241], [2.434099363908185]]
[[2.3089556694695355], [2.367587314679404], [2.3864034110286694], [2.3709884573644624]]


In [47]:
texSource

'../autotikz/'

In [4]:
final_dict_oct_info

{'atom_dist_max': 0.05321132609372134,
 'devi_linear_avrg': 0,
 'devi_linear_max': 0,
 'dist_del_all': 0.1251436944386497,
 'dist_del_eq': 0.018816096349265177,
 'max_del_sig_angle': 7.2737722660320401,
 'num_coord_metal': 6,
 'oct_angle_devi_max': 4.608051083150329,
 'rmsd_max': 0.05321132609372109}

In [7]:
init_eq_dent

[1]